In [1]:
import numpy as np
import pickle
import glob
import matplotlib.pyplot as plt
import pandas as pd
from scipy import *
import os
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout,Conv1D,MaxPooling1D,Flatten
from sklearn.model_selection import train_test_split
from sklearn import *
from sklearn.metrics import *
sns.set()
import warnings
warnings.filterwarnings("ignore")
length = 277

2024-03-19 21:57:17.898923: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-19 21:57:17.952370: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-19 21:57:17.952409: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-19 21:57:17.954561: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-19 21:57:17.970685: I tensorflow/core/platform/cpu_feature_guar

In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6075727844423715116
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5803868160
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4108286304210329175
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 4070 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9"
xla_global_id: 416903419
]


2024-03-19 21:57:20.981569: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 21:57:21.114377: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 21:57:21.114422: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 21:57:21.443885: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 21:57:21.443929: I external/local_xla/xla/stream_executor

In [3]:
# Load the training and testing data:
train_values = np.empty(shape=[0, length])
test_values = np.empty(shape=[0, length])

train_beats = glob.glob('./data/train_test/train_patients.csv')
test_beats = glob.glob('./data/train_test/test_patients.csv')

for j in train_beats:
    print('Loading ', j)
    csvrows = np.loadtxt(j, delimiter=',')
    train_values = np.append(train_values, csvrows, axis=0)

for j in test_beats:
    print('Loading ', j)
    csvrows = np.loadtxt(j, delimiter=',')
    test_values = np.append(test_values, csvrows, axis=0)
    
print(train_values.shape)
print(test_values.shape)

# Separate the training and testing data, and one-hot encode Y:
X_train = train_values[:,:-2]
X_test = test_values[:,:-2]
y_train = train_values[:,-2]
y_test = test_values[:,-2]
X_test1 = X_test.reshape(-1, X_train.shape[1], 1)
y_test1=to_categorical(y_test)


X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42, stratify=y_train)

print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("y_train shape:", y_train.shape)
print("y_val shape:", y_val.shape)
X_train1 = X_train.reshape(-1, X_train.shape[1], 1)
y_train1=to_categorical(y_train)

X_val1 = X_val.reshape(-1, X_train.shape[1], 1)
y_val1=to_categorical(y_val)


Loading  ./data/train_test/train_patients.csv
Loading  ./data/train_test/test_patients.csv
(38752, 277)
(72795, 277)
X_train shape: (31001, 275)
X_val shape: (7751, 275)
y_train shape: (31001,)
y_val shape: (7751,)


In [4]:
# Initialize the model
cnnmodel = Sequential()

# Ensure the input_shape parameter matches the shape of X_train1
cnnmodel.add(Conv1D(396, 5, strides=1, activation='relu', input_shape=(X_train1.shape[1],X_train1.shape[2])))
cnnmodel.add(MaxPooling1D(pool_size=2, strides=2))

cnnmodel.add(Conv1D(192, 7, strides=1, activation='relu'))

cnnmodel.add(MaxPooling1D(pool_size=2, strides=2))
cnnmodel.add(Conv1D(88, 9, strides=1, activation='relu'))
cnnmodel.add(MaxPooling1D(pool_size=2, strides=2))
cnnmodel.add(Conv1D(34, 11, strides=1, activation='relu'))
cnnmodel.add(MaxPooling1D(pool_size=2, strides=2))

# Flatten the tensor output from the convolutional part of the CNN
cnnmodel.add(Flatten())

# Add fully connected layers
cnnmodel.add(Dense(30, activation='relu'))
cnnmodel.add(Dense(20, activation='relu'))
cnnmodel.add(Dense(7, activation='softmax'))  

cnnmodel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 271, 396)          2376      
                                                                 
 max_pooling1d (MaxPooling1  (None, 135, 396)          0         
 D)                                                              
                                                                 
 conv1d_1 (Conv1D)           (None, 129, 192)          532416    
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 64, 192)           0         
 g1D)                                                            
                                                                 
 conv1d_2 (Conv1D)           (None, 56, 88)            152152    
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 28, 88)            0

2024-03-19 21:57:22.952209: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 21:57:22.952312: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 21:57:22.952347: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 21:57:22.952513: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-19 21:57:22.952542: I external/local_xla/xla/stream_executor

In [5]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)


In [6]:
class_weights

array([0.25926205, 2.33370973, 2.33265613, 2.33370973, 2.33370973,
       2.33265613])

In [7]:
cnnmodel.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10, mode='min',min_delta=0.001, restore_best_weights=True)
history = cnnmodel.fit(X_train1, y_train1, validation_data=(X_val, y_val1), epochs=50,callbacks=[early_stopping], verbose=1)


Epoch 1/50


2024-03-19 21:57:23.938599: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-03-19 21:57:24.063779: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-19 21:57:24.293713: I external/local_tsl/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2024-03-19 21:57:24.671878: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f6e5cd179a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-19 21:57:24.671910: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 4070 Laptop GPU, Compute Capability 8.9
2024-03-19 21:57:24.675680: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1710885444.727435  316085 devic

969/969 [==============================] - 11s 9ms/step - loss: 0.2857 - accuracy: 0.9123 - val_loss: 0.1664 - val_accuracy: 0.9489
Epoch 2/50
969/969 [==============================] - 7s 8ms/step - loss: 0.1101 - accuracy: 0.9669 - val_loss: 0.0837 - val_accuracy: 0.9760
Epoch 3/50
969/969 [==============================] - 7s 7ms/step - loss: 0.0758 - accuracy: 0.9766 - val_loss: 0.0677 - val_accuracy: 0.9795
Epoch 4/50
969/969 [==============================] - 7s 7ms/step - loss: 0.0608 - accuracy: 0.9801 - val_loss: 0.0631 - val_accuracy: 0.9822
Epoch 5/50
969/969 [==============================] - 7s 7ms/step - loss: 0.0523 - accuracy: 0.9838 - val_loss: 0.0633 - val_accuracy: 0.9790
Epoch 6/50
969/969 [==============================] - 7s 8ms/step - loss: 0.0435 - accuracy: 0.9859 - val_loss: 0.0569 - val_accuracy: 0.9816
Epoch 7/50
969/969 [==============================] - 24s 25ms/step - loss: 0.0338 - accuracy: 0.9891 - val_loss: 0.0494 - val_accuracy: 0.9857
Epoch 8/50
969

In [8]:
class_counts = np.bincount(y_test.astype(int))
print(class_counts)


[    0 58952  2493  6725  2923  1277   425]


In [9]:
from sklearn.metrics import confusion_matrix, f1_score

# Predict the test data
y_pred = cnnmodel.predict(X_test1)

# Convert the predicted probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

# Create the confusion matrix
cm = confusion_matrix(y_test, y_pred_labels)
print("Confusion Matrix:")
print(cm)

# Calculate the F1 score
f1 = f1_score(y_test, y_pred_labels, average='weighted')
print("F1 Score:", f1)




2275/2275 [==============================] - 5s 2ms/step
Confusion Matrix:
[[58560    11     8   161    80   132]
 [    0  2492     0     1     0     0]
 [   16     0  6647     6    50     6]
 [   27     1     8  2865     2    20]
 [   84     0     4     6  1182     1]
 [    0     0     2     3     0   420]]
F1 Score: 0.9915758400785036


In [10]:
cnnmodel.save('./models/cnnmodel.h5')
